In [1]:
import pandas as pd 
import numpy as np

In [2]:
crops = pd.read_csv("../selected_crops_temp.csv")
crops

,CROP TYPE,SOIL TYPE,REGION,WEATHER CONDITION,TEMP_MIN,TEMP_MAX,WATER REQUIREMENT
0,POTATO,DRY,DESERT,NORMAL,10,20,8.500
1,POTATO,DRY,DESERT,SUNNY,10,20,10.000
2,POTATO,DRY,DESERT,WINDY,10,20,9.500
3,POTATO,DRY,DESERT,RAINY,10,20,0.500
4,POTATO,DRY,DESERT,NORMAL,20,30,9.500
...,...,...,...,...,...,...,...
763,ONION,WET,HUMID,RAINY,30,40,0.100
764,ONION,WET,HUMID,NORMAL,40,50,4.625
765,ONION,WET,HUMID,SUNNY,40,50,6.125
766,ONION,WET,HUMID,WINDY,40,50,5.625


In [3]:
wheat = crops[crops["CROP TYPE"] == "WHEAT"]

In [4]:
wheat

,CROP TYPE,SOIL TYPE,REGION,WEATHER CONDITION,TEMP_MIN,TEMP_MAX,WATER REQUIREMENT
192,WHEAT,DRY,DESERT,NORMAL,10,20,10.54
193,WHEAT,DRY,DESERT,SUNNY,10,20,12.00
194,WHEAT,DRY,DESERT,WINDY,10,20,11.54
195,WHEAT,DRY,DESERT,RAINY,10,20,2.54
196,WHEAT,DRY,DESERT,NORMAL,20,30,11.50
...,...,...,...,...,...,...,...
379,WHEAT,WET,HUMID,RAINY,30,40,0.20
380,WHEAT,WET,HUMID,NORMAL,40,50,5.48
381,WHEAT,WET,HUMID,SUNNY,40,50,6.98
382,WHEAT,WET,HUMID,WINDY,40,50,6.48


In [5]:
wheat.info

<bound method DataFrame.info of     CROP TYPE SOIL TYPE  REGION WEATHER CONDITION  TEMP_MIN  TEMP_MAX  \
192     WHEAT       DRY  DESERT            NORMAL        10        20   
193     WHEAT       DRY  DESERT             SUNNY        10        20   
194     WHEAT       DRY  DESERT             WINDY        10        20   
195     WHEAT       DRY  DESERT             RAINY        10        20   
196     WHEAT       DRY  DESERT            NORMAL        20        30   
..        ...       ...     ...               ...       ...       ...   
379     WHEAT       WET   HUMID             RAINY        30        40   
380     WHEAT       WET   HUMID            NORMAL        40        50   
381     WHEAT       WET   HUMID             SUNNY        40        50   
382     WHEAT       WET   HUMID             WINDY        40        50   
383     WHEAT       WET   HUMID             RAINY        40        50   

     WATER REQUIREMENT  
192              10.54  
193              12.00  
194             

In [6]:
wheat = wheat.drop("CROP TYPE", axis = 1)

In [7]:
wheat = wheat.reset_index()

In [8]:
wheat.isnull().sum()

index                0
SOIL TYPE            0
REGION               0
WEATHER CONDITION    0
TEMP_MIN             0
TEMP_MAX             0
WATER REQUIREMENT    0
dtype: int64

In [9]:
X = wheat.drop(["WATER REQUIREMENT", "index"], axis = 1)

In [10]:
y = wheat["WATER REQUIREMENT"]

In [11]:
X

,SOIL TYPE,REGION,WEATHER CONDITION,TEMP_MIN,TEMP_MAX
0,DRY,DESERT,NORMAL,10,20
1,DRY,DESERT,SUNNY,10,20
2,DRY,DESERT,WINDY,10,20
3,DRY,DESERT,RAINY,10,20
4,DRY,DESERT,NORMAL,20,30
...,...,...,...,...,...
187,WET,HUMID,RAINY,30,40
188,WET,HUMID,NORMAL,40,50
189,WET,HUMID,SUNNY,40,50
190,WET,HUMID,WINDY,40,50


In [12]:
y

0      10.54
1      12.00
2      11.54
3       2.54
4      11.50
       ...  
187     0.20
188     5.48
189     6.98
190     6.48
191     0.30
Name: WATER REQUIREMENT, Length: 192, dtype: float64

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [14]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

one = OneHotEncoder(handle_unknown = "ignore", sparse = False)
categorical_features = ["SOIL TYPE", "REGION", "WEATHER CONDITION"]
transformer = ColumnTransformer([("one", one, categorical_features)], remainder = "passthrough")

transformed_X_train = transformer.fit_transform(X_train)
transformed_X_train

C:\Users\madan\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[ 0.,  1.,  0., ...,  1., 40., 50.],
       [ 0.,  0.,  1., ...,  0., 40., 50.],
       [ 0.,  0.,  1., ...,  0., 10., 20.],
       ...,
       [ 1.,  0.,  0., ...,  0., 10., 20.],
       [ 0.,  0.,  1., ...,  1., 20., 30.],
       [ 0.,  1.,  0., ...,  0., 40., 50.]])

In [15]:
transformed_X_train.shape

(153, 13)

In [16]:
X["SOIL TYPE"].value_counts()

DRY      64
HUMID    64
WET      64
Name: SOIL TYPE, dtype: int64

In [17]:
transformed_X_test = transformer.transform(X_test)

In [18]:
transformed_X_test.shape

(39, 13)

In [19]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(transformed_X_train, y_train)
model.score(transformed_X_test, y_test)*100

97.18305525858361

In [20]:
y_preds = model.predict(transformed_X_test)
y_preds

array([ 2.8715, 10.969 , 11.4215,  6.851 , 11.2825,  6.3297,  1.6641,
        6.0509,  5.7291,  2.5856, 11.24  ,  7.1715,  8.9954,  0.294 ,
        0.343 ,  8.864 ,  8.8693,  4.1609,  0.6234, 11.2466,  8.1416,
       12.707 ,  1.1471, 11.5471,  7.6647,  7.7189,  7.2566,  8.157 ,
       10.1362, 12.0455, 10.8207,  9.6873,  1.6276,  8.4347,  9.0956,
       11.9407,  8.6953, 12.192 , 11.084 ])

In [21]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_preds)
mse

0.4387789733333338

In [22]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_preds)
mae

0.5660358974358979

In [23]:
from sklearn.metrics import r2_score
score = r2_score(y_test, y_preds)
score

0.9718305525858361

In [24]:
adjusted_r2 = 1 - (1-score)*(len(y_test)-1)/(len(y_test)-transformed_X_test.shape[1]-1)
adjusted_r2

0.9571824399304708

In [25]:
## with linear regression
from sklearn.linear_model import Ridge

ridgeModel = Ridge()
ridgeModel.fit(transformed_X_train, y_train)
ridgeModel.score(transformed_X_test, y_test)*100

97.27081492687365

In [26]:
## with SVM
from sklearn.svm import SVR
svmModel = SVR()
svmModel.fit(transformed_X_train, y_train)
svmModel.score(transformed_X_test, y_test)*100

4.123899873339665

In [27]:
## with nuSVM
from sklearn.svm import NuSVR
nuSVMModel = NuSVR()
nuSVMModel.fit(transformed_X_train, y_train)
nuSVMModel.score(transformed_X_test, y_test)

-0.023935759902197207

In [28]:
temp = ["DRY", "DESERT", "WINDY", 35, 55]

In [29]:
temp = np.array(temp)
temp

array(['DRY', 'DESERT', 'WINDY', '35', '55'], dtype='<U11')

In [30]:
temp = temp.reshape(1, -1)
temp.shape

(1, 5)

In [31]:
temp_df = pd.DataFrame(temp, columns = X.columns )
val = temp_df["SOIL TYPE"][0]
type(val)

str

In [32]:
transformed_temp = transformer.transform(temp_df)
transformed_temp.shape

(1, 13)

In [33]:
X.columns

Index(['SOIL TYPE', 'REGION', 'WEATHER CONDITION', 'TEMP_MIN', 'TEMP_MAX'], dtype='object')

In [34]:
temp_preds = model.predict(transformed_temp)
temp_preds

array([13.7747])

In [35]:
## pickling 
import pickle

In [36]:
pickle.dump(model, open("wheat_model.pkl", "wb"))

In [37]:
pickle.dump(transformer, open("wheat_transformer.pkl", "wb"))